# Introduction,  and 
The 2023 Kidney and Kidney Tumor Segmentation Challenge 

## Motivation



## Problem Statement 


# Imports segments
import module torch, numpy, .....

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Data Sources 
Data sources used, clearly documented and described. Enough detail is provided for the data to be found again.


In [3]:
# import data here


# Exploratory Analysis of Data
Provide details about the properties, number of classes, pre-processing, challenging aspects, etc. of the data




# Design of Neural network etc

## Exiting Neural Network

### Which part of the Neural Network will be freezed
### Design of neural network for the specific purpose


In [4]:
# This is sample only
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [5]:

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
